In [1]:
//Australia having very distinct climate zones,no single model will work for all australia
//Different models need to be trained for different locations
//In this approach,separate models will be created for different location and saved in the directory
//for the prediction particular location model will be loaded and prediction will be done
//Data used was weatherAUS_YN.csv(before imputing the  null values) as it was  observed  that 
//dropping null values are more effective than imputing (In our scenario imputing means blunting the features i.e.  generalising the data)
//Wind direction (categorical columns) not tested or used in the modelling as too much of encoding required and crashing the docker
//correlation matrix(wrangling file)& trial & Error methods were used to select the features)

Intitializing Scala interpreter ...

Spark Web UI available at http://6183b873be90:4040
SparkContext available as 'sc' (version = 2.4.5, master = local[*], app id = local-1591528110560)
SparkSession available as 'spark'


2020-06-07 11:08:26,432 WARN  [main] util.NativeCodeLoader (NativeCodeLoader.java:<clinit>(62)) - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Import required libraries

In [2]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator  
import org.apache.spark.mllib.evaluation.MulticlassMetrics  
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics  
import org.apache.spark.ml.classification.LogisticRegression  
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit, CrossValidator}  
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer, OneHotEncoderEstimator,VectorIndexer}  
import org.apache.spark.ml.linalg.Vectors  
import org.apache.spark.ml.{Pipeline,PipelineModel}  
import org.apache.log4j._  
Logger.getLogger("org").setLevel(Level.ERROR)  

import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit, CrossValidator}
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer, OneHotEncoderEstimator, VectorIndexer}
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.log4j._


# Starting spark session

In [3]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@a8f6d51


# Importing the file

In [4]:
val data = spark.read.option("header","true").
    option("inferSchema","true").
    format("csv").
    load("/home/weatherAUS_YN.csv")

data: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 22 more fields]


In [5]:
val df= data.withColumn("Month",month(data("Date")))

df: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 23 more fields]


#  Selecting required fields with right data type

In [6]:
val df1 = df.selectExpr("Date","Location",
                        //"cast(MinTemp as double) MinTemp",
                        "cast (MaxTemp as double) MaxTemp",
                       // "cast (Rainfall as double) Rainfall",
                       // "cast (Evaporation as double) Evaporation",
                        // "cast (Sunshine as double) Sunshine",
                        //"WindGustDir",
                        // "cast (WindGustSpeed as double) WindGustSpeed",
                        //"WindDir9am","WindDir3pm",
                         //"cast (WindSpeed9am as double) WindSpeed9am",
                         //"cast (WindSpeed3pm as double) WindSpeed3pm",
                         //"cast (Humidity9am as double) Humidity9am",
                          "cast (Humidity3pm as double) Humidity3pm", 
                         //"cast (Pressure9am as double) Pressure9am",
                         "cast (Pressure3pm as double) Pressure3pm",
                         //"cast (Cloud9am as double) Cloud9am",
                         //"cast (Cloud3pm as double) Cloud3pm",
                        "cast (Temp3pm as double) Temp3pm",
                        "cast (Temp9am as double) Temp9am",
                         "cast (RainToday as integer) RainToday",
                         "cast (Month as String) Month",
                         "cast (RainTomorrow as integer) RainTomorrow"
                      
                                                  )
                        

df1: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 8 more fields]


In [7]:
df1.columns

res2: Array[String] = Array(Date, Location, MaxTemp, Humidity3pm, Pressure3pm, Temp3pm, Temp9am, RainToday, Month, RainTomorrow)


# Filtering and forming a particular location data

In [10]:
val LocData=df1.filter($"Location"==="Perth")toDF()

LocData: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 8 more fields]


In [11]:
LocData.count()

res4: Long = 3193


# Dropping the null values

In [12]:
val Loc =LocData.na.drop()

Loc: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 8 more fields]


In [13]:
Loc.count()

res5: Long = 3184


# Setting the label column(Rain tomorrow)

In [14]:
val df2 = (Loc.select(Loc("RainTomorrow").as("label"), 
           $"Location",
           $"Date",
          //$"MinTemp",
          $"MaxTemp", 
         $"Month",
         $"Raintoday",
         //$"WindDir3pm",
         //$"WindSpeed9am",
         // $"WindSpeed3pm", 
         //$"Humidity9am",
          $"Humidity3pm",
          //$"Pressure9am",
          //$"Cloud3pm", 
          $"Pressure3pm", 
           $"Temp9am", 
          $"Temp3pm"))

df2: org.apache.spark.sql.DataFrame = [label: int, Location: string ... 8 more fields]


# Indexing the month column(categorical)-To capture the seasonal pattern

In [15]:
val Indexer=new StringIndexer().setInputCol("Month").setOutputCol("MonIn")




Indexer: org.apache.spark.ml.feature.StringIndexer = strIdx_ae36a46113bc


# Encoding the MonIn( column

In [16]:
val Encoder=new OneHotEncoderEstimator().setInputCols(Array("MonIn")).setOutputCols(Array("MonEn"))


Encoder: org.apache.spark.ml.feature.OneHotEncoderEstimator = oneHotEncoder_f1befb92b0e8


# Setting the assembler

In [17]:
 val assembler=(new VectorAssembler().setInputCols(Array(//"MinTemp",
                                                        "Raintoday",
                                                         "MaxTemp",
                                                        "Humidity3pm",
                                                         "Temp3pm",
                                                         "MonEn",
                                                        //"Humidity9am",
                                                        //"WindSpeed3pm",
                                                        "Pressure3pm"
                                                        )).setOutputCol("features"))

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_cc24206c4f2e


# Splitting the data into test and train sets

In [18]:
val Array(training,test)=df2.randomSplit(Array(0.7,0.3),seed=999)

training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: int, Location: string ... 8 more fields]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: int, Location: string ... 8 more fields]


# Logistic regression

In [19]:
val lr=new LogisticRegression().setMaxIter(30).setThreshold(0.50)
       

lr: org.apache.spark.ml.classification.LogisticRegression = logreg_0ffe5fc0ad75


# Setting the pipeline

In [20]:
val pipeline=new Pipeline().setStages(Array(Indexer,Encoder,assembler,lr))

pipeline: org.apache.spark.ml.Pipeline = pipeline_8ae362c04e98


# Training the model on training data

In [21]:
val model1=pipeline.fit(training)

2020-06-07 11:11:42,905 WARN  [Thread-4] netlib.BLAS (BLAS.java:<clinit>(61)) - Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2020-06-07 11:11:42,909 WARN  [Thread-4] netlib.BLAS (BLAS.java:<clinit>(61)) - Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


model1: org.apache.spark.ml.PipelineModel = pipeline_8ae362c04e98


# transforming test data with model

In [22]:
val results =model1.transform(test)

results: org.apache.spark.sql.DataFrame = [label: int, Location: string ... 14 more fields]


# Model Evaluation

In [23]:
val predictions=results.select($"prediction",$"label").as[(Double,Double)].rdd

predictions: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[109] at rdd at <console>:38


In [24]:
val metrics= new MulticlassMetrics(predictions)

metrics: org.apache.spark.mllib.evaluation.MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@57c85456


In [25]:
println(metrics.confusionMatrix)

747.0  35.0  
108.0  82.0  


(1,1) no  pred no  rain
(2,2) yes pred yes rain
(1,2) no pred yes rain 
(2,1)  yes pred no rain 

In [26]:
metrics.accuracy

res7: Double = 0.852880658436214


# Saving the model

In [27]:
model1.write.overwrite().save("/home/model_Perth")

# Saving the pipeline

In [28]:
pipeline.write.overwrite().save("/home/pipeline_Perth")

# Reloading the model

In [29]:
val modelLoaded=PipelineModel.load("/home/model_Perth")

modelLoaded: org.apache.spark.ml.PipelineModel = pipeline_8ae362c04e98


# Using the model

In [30]:
val dfu= modelLoaded.transform(df2)

dfu: org.apache.spark.sql.DataFrame = [label: int, Location: string ... 14 more fields]


In [31]:
df2.show()

+-----+--------+-------------------+-------+-----+---------+-----------+-----------+-------+-------+
|label|Location|               Date|MaxTemp|Month|Raintoday|Humidity3pm|Pressure3pm|Temp9am|Temp3pm|
+-----+--------+-------------------+-------+-----+---------+-----------+-----------+-------+-------+
|    0|   Perth|2008-07-01 00:00:00|   18.8|    7|        0|       53.0|     1024.5|    8.5|   18.1|
|    0|   Perth|2008-07-02 00:00:00|   20.7|    7|        0|       39.0|     1019.0|   11.1|   19.7|
|    1|   Perth|2008-07-03 00:00:00|   19.9|    7|        0|       71.0|     1015.6|   12.1|   17.7|
|    1|   Perth|2008-07-04 00:00:00|   19.2|    7|        1|       73.0|     1018.4|   13.2|   17.7|
|    1|   Perth|2008-07-05 00:00:00|   16.4|    7|        1|       57.0|     1022.1|   15.9|   16.0|
|    0|   Perth|2008-07-06 00:00:00|   15.9|    7|        1|       41.0|     1029.6|    6.9|   15.5|
|    0|   Perth|2008-07-07 00:00:00|   18.3|    7|        0|       36.0|     1024.2|    8.7

In [32]:
val testev=dfu.select("Location","Date","RawPrediction","probability","label","prediction").toDF()

testev: org.apache.spark.sql.DataFrame = [Location: string, Date: timestamp ... 4 more fields]


In [33]:
testev.show()

+--------+-------------------+--------------------+--------------------+-----+----------+
|Location|               Date|       RawPrediction|         probability|label|prediction|
+--------+-------------------+--------------------+--------------------+-----+----------+
|   Perth|2008-07-01 00:00:00|[0.82942927553653...|[0.69623423961535...|    0|       0.0|
|   Perth|2008-07-02 00:00:00|[2.15246309691057...|[0.89589871976251...|    0|       0.0|
|   Perth|2008-07-03 00:00:00|[-0.9608150338322...|[0.27671504120268...|    1|       1.0|
|   Perth|2008-07-04 00:00:00|[-1.7214488860469...|[0.15168463210551...|    1|       1.0|
|   Perth|2008-07-05 00:00:00|[-0.1847097658736...|[0.45395340114355...|    1|       1.0|
|   Perth|2008-07-06 00:00:00|[1.37035487013528...|[0.79743748212742...|    0|       0.0|
|   Perth|2008-07-07 00:00:00|[2.44907996692623...|[0.92049414453850...|    0|       0.0|
|   Perth|2008-07-08 00:00:00|[1.87014233994295...|[0.86647474652492...|    1|       0.0|
|   Perth|